# Advanced Machine Learning for NLP and Text Processing 
## Project 1 : OpenFoodFacts
### Yanis FADILI & Lies HAOUAS

## Importation des libriairies

In [173]:
import pandas as pd
import numpy as np
import math 
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import average_precision_score as precision
import seaborn as sns
np.random.seed(271099)

## Importation du dataset

In [174]:
df1 = pd.read_csv('/content/sample_data/data1.csv',sep="\t",index_col=0,encoding='utf-8')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,14,28,29,30,32,53) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [175]:
#df2 = pd.read_csv('/content/sample_data/data2.csv',sep="\t",index_col=0,encoding='utf-8')

In [176]:
#df3 = pd.read_csv('/content/sample_data/data3.csv',sep="\t",index_col=0,encoding='utf-8')

In [177]:
#On fusionne les 3 datasets qu'on vient de lire pour avoir un dataset plus grand 100 000 lignes
df = pd.concat([df1])

Aperçu des premières lignes du dataset initial

In [178]:
df.head(5)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,...,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,jeunes pousses,NaN,NaN,NaN,NaN,NaN,NaN,endives,endives,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,en:france,France,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1630483911,2021-09-01T08:11:51Z,1630484064,2021-09-01T08:14:24Z,L.casei,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spain,en:spain,Spain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spain,en:spain,Spain,"Leche semidesnatada, azucar 6.9% leche desnata...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,NaN,130 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000000000003327986,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1574175736,2019-11-19T15:02:16Z,1624390765,2021-06-22T19:39:25Z,Filetes de pollo empanado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Espagne,en:spain,Spain,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [179]:
df.shape

(50000, 186)

##Exploration du dataset

In [180]:
totalRow = df.shape[0]
#150 000

In [181]:
#On regarde les différents types qu'on a dans le dataset
df.dtypes.unique()

array([dtype('O'), dtype('int64'), dtype('float64')], dtype=object)

In [182]:
col = df.columns.tolist()

Description et calcul des valeurs remarquables (moyenne, max, quartiles) pour chaque variable

In [183]:
for index,value in enumerate(df.describe()):
    print(col[index].upper(),df[value].describe(),sep='\n')

CODE
count    5.000000e+04
mean     1.546501e+09
std      5.388986e+07
min      1.335538e+09
25%      1.489076e+09
50%      1.574015e+09
75%      1.587665e+09
max      1.634083e+09
Name: created_t, dtype: float64
URL
count    5.000000e+04
mean     1.590064e+09
std      2.677677e+07
min      1.416874e+09
25%      1.587584e+09
50%      1.587645e+09
75%      1.595084e+09
max      1.634083e+09
Name: last_modified_t, dtype: float64
CREATOR
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: abbreviated_product_name, dtype: float64
CREATED_T
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: cities, dtype: float64
CREATED_DATETIME
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: allergens_en, dtype: float64
LAST_MODIFIED_T
count    37447.000000
mean        84.632737
std        209.586509
min          0.000000
25%    

Nombre d'éléments nuls pour chaque colonne

In [184]:
sumNAbyColumn = df.isna().sum()
sumNAbyColumn

code                      0
url                       0
creator                   0
created_t                 0
created_datetime          0
                      ...  
choline_100g          49999
phylloquinone_100g    49855
beta-glucan_100g      50000
inositol_100g         49999
carnitine_100g        50000
Length: 186, dtype: int64

Suppression des colonnes totalement vides

In [185]:
df = df.drop(df.columns[df.apply(lambda col: col.isna().sum() == totalRow)], axis=1)

Suppression des colonnées peu ou pas pertientes dans le cadre du TD

In [186]:
df = df.drop(['code','url','image_url','creator','created_t','created_datetime','last_modified_t','last_modified_datetime','image_small_url','categories_tags',
             'categories_en','countries_tags','countries_en','additives_tags','additives_en','states_tags','states_en','main_category_en','brands_tags'], axis=1)

In [187]:
df.shape

(50000, 120)

Nom des colonnes

In [188]:
newCol = df.columns.tolist()


In [189]:
#Colonne supprimée
list(set(col) - set(newCol))

['-capric-acid_100g',
 'created_t',
 'image_url',
 'states_en',
 'casein_100g',
 'image_small_url',
 '-maltodextrins_100g',
 'additives_tags',
 'carnitine_100g',
 '-eicosapentaenoic-acid_100g',
 'states_tags',
 '-maltose_100g',
 'allergens_en',
 'last_modified_datetime',
 '-lignoceric-acid_100g',
 '-cerotic-acid_100g',
 '-gondoic-acid_100g',
 'ingredients_that_may_be_from_palm_oil',
 '-montanic-acid_100g',
 'no_nutriments',
 '-gamma-linolenic-acid_100g',
 'code',
 '-nervonic-acid_100g',
 'collagen-meat-protein-ratio_100g',
 'cities',
 'nutrition-score-uk_100g',
 'silica_100g',
 'main_category_en',
 '-sucrose_100g',
 '-caprylic-acid_100g',
 '-myristic-acid_100g',
 'serum-proteins_100g',
 'last_modified_t',
 'chlorophyl_100g',
 'omega-9-fat_100g',
 'countries_tags',
 'countries_en',
 'created_datetime',
 '-butyric-acid_100g',
 'beta-glucan_100g',
 'carbon-footprint_100g',
 '-fructose_100g',
 '-mead-acid_100g',
 '-caproic-acid_100g',
 '-glucose_100g',
 'additives_en',
 '-lauric-acid_100g'

Nombre de colonnes supprimé

In [190]:
len(col)-len(newCol)

66

On voit qu'il reste toujours des colonnes avec des lignes avec des valeurs non renseignées



In [191]:
for index,value in enumerate(df.isna().sum()):
    print(newCol[index],value)

product_name 1009
generic_name 49270
quantity 46977
packaging 47824
packaging_tags 47825
packaging_text 49981
brands 23180
categories 13878
origins 49614
origins_tags 49615
origins_en 49615
manufacturing_places 49362
manufacturing_places_tags 49362
labels 46557
labels_tags 46557
labels_en 46557
emb_codes 49820
emb_codes_tags 49820
first_packaging_code_geo 49953
cities_tags 49950
purchase_places 49051
stores 46293
countries 84
ingredients_text 11295
allergens 41529
traces 49507
traces_tags 49346
traces_en 49346
serving_size 12529
serving_quantity 12553
additives_n 11295
ingredients_from_palm_oil_n 11295
ingredients_from_palm_oil_tags 49937
ingredients_that_may_be_from_palm_oil_n 11295
ingredients_that_may_be_from_palm_oil_tags 47993
nutriscore_score 21072
nutriscore_grade 21072
nova_group 13788
pnns_groups_1 3
pnns_groups_2 3
states 0
brand_owner 16730
ecoscore_score_fr 41523
ecoscore_grade_fr 41523
main_category 13878
image_ingredients_url 44471
image_ingredients_small_url 44471
image_

In [192]:
df.shape

(50000, 120)

Calcul du seuil de données non-nulles égales à 80% du dataset

In [193]:
treshold = round((df.shape[0] * 95) / 100)

In [194]:
treshold

47500

Suppression des colonnes ayant plus de 80% de lignes vides

In [195]:
df = df.drop(df.columns[df.apply(lambda col: col.isna().sum() > treshold)], axis=1)

In [196]:
df.columns.tolist()

['product_name',
 'quantity',
 'brands',
 'categories',
 'labels',
 'labels_tags',
 'labels_en',
 'stores',
 'countries',
 'ingredients_text',
 'allergens',
 'serving_size',
 'serving_quantity',
 'additives_n',
 'ingredients_from_palm_oil_n',
 'ingredients_that_may_be_from_palm_oil_n',
 'nutriscore_score',
 'nutriscore_grade',
 'nova_group',
 'pnns_groups_1',
 'pnns_groups_2',
 'states',
 'brand_owner',
 'ecoscore_score_fr',
 'ecoscore_grade_fr',
 'main_category',
 'image_ingredients_url',
 'image_ingredients_small_url',
 'image_nutrition_url',
 'image_nutrition_small_url',
 'energy-kcal_100g',
 'energy_100g',
 'fat_100g',
 'saturated-fat_100g',
 'monounsaturated-fat_100g',
 'polyunsaturated-fat_100g',
 'trans-fat_100g',
 'cholesterol_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'fiber_100g',
 'proteins_100g',
 'salt_100g',
 'sodium_100g',
 'vitamin-a_100g',
 'vitamin-c_100g',
 'vitamin-b1_100g',
 'vitamin-pp_100g',
 'potassium_100g',
 'calcium_100g',
 'iron_100g',
 'nutrition-score-

Colonnes supprimées

In [197]:
list(set(newCol) - set(df.columns.tolist()))

['packaging_text',
 'origins',
 'energy-from-fat_100g',
 'biotin_100g',
 '-docosahexaenoic-acid_100g',
 'origins_en',
 '-soluble-fiber_100g',
 'choline_100g',
 'alcohol_100g',
 'ingredients_from_palm_oil_tags',
 'folates_100g',
 'copper_100g',
 'fruits-vegetables-nuts_100g',
 'inositol_100g',
 '-arachidonic-acid_100g',
 'fluoride_100g',
 'omega-6-fat_100g',
 'omega-3-fat_100g',
 'emb_codes_tags',
 'origins_tags',
 'cities_tags',
 'vitamin-b9_100g',
 'manufacturing_places',
 'zinc_100g',
 '-stearic-acid_100g',
 'emb_codes',
 'fruits-vegetables-nuts-estimate_100g',
 'chromium_100g',
 '-lactose_100g',
 'vitamin-k_100g',
 'manufacturing_places_tags',
 'chloride_100g',
 'pantothenic-acid_100g',
 'phosphorus_100g',
 '-behenic-acid_100g',
 'polyols_100g',
 'taurine_100g',
 'bicarbonate_100g',
 '-linoleic-acid_100g',
 'first_packaging_code_geo',
 'manganese_100g',
 'traces_tags',
 'traces',
 'iodine_100g',
 'packaging_tags',
 '-alpha-linolenic-acid_100g',
 'nucleotides_100g',
 'energy-kj_100g'

Colonnes conservées

In [198]:
df.sample(5)

,product_name,quantity,brands,categories,labels,labels_tags,labels_en,stores,countries,ingredients_text,allergens,serving_size,serving_quantity,additives_n,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,brand_owner,ecoscore_score_fr,ecoscore_grade_fr,main_category,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,vitamin-a_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-pp_100g,potassium_100g,calcium_100g,iron_100g,nutrition-score-fr_100g
41464,White enriched brown'n serve rolls,NaN,NaN,"Plant-based foods and beverages, Plant-based f...",NaN,NaN,NaN,NaN,United States,"Unbleached enriched wheat flour (wheat flour, ...",en:gluten,1 ROLL (38 g),38.0,7.0,0.0,0.0,1.0,b,4.0,Cereals and potatoes,Bread,"en:to-be-completed, en:nutrition-facts-complet...","Safeway, Inc.",NaN,NaN,en:breads,NaN,NaN,NaN,NaN,237.0,992.0,3.95,0.00,NaN,NaN,0.0,0.0,44.74,2.63,2.6,7.89,1.1850,0.474,NaN,NaN,0.001,0.005263,0.079,0.105,0.00789,1.0
16458,Unsweetened vanilla cold brew 100% arabica cof...,NaN,NaN,Beverages,NaN,NaN,NaN,NaN,United States,"Water, 100% arabica coffee, natural flavor.",NaN,1 BOTTLE (325 ml),325.0,0.0,0.0,0.0,1.0,b,4.0,Beverages,Unsweetened beverages,"en:to-be-completed, en:nutrition-facts-complet...",WIDE AWAKE COFFEE CO.,NaN,NaN,en:beverages,NaN,NaN,NaN,NaN,2.0,8.0,0.00,0.00,NaN,NaN,0.0,0.0,0.00,0.00,0.0,0.31,0.0425,0.017,NaN,NaN,NaN,NaN,NaN,0.000,0.00000,1.0
28117,Dressing,NaN,Redner's Warehouse Markets,"Groceries, Sauces, Salad dressings",NaN,NaN,NaN,NaN,United States,"Sugar, vinegar (distilled and cider), water, v...",NaN,2 Tbsp (30 ml),30.0,4.0,0.0,0.0,14.0,d,4.0,Fat and sauces,Dressings and sauces,"en:to-be-completed, en:nutrition-facts-complet...",REDNER'S WAREHOUSE MARKETS,29.0,d,en:salad-dressings,NaN,NaN,NaN,NaN,267.0,1117.0,15.00,1.67,NaN,NaN,0.0,0.0,36.67,33.33,0.0,0.00,0.8325,0.333,0.0,0.0,NaN,NaN,NaN,0.000,0.00000,14.0
29622,Nat vly rstd nt crnch bars almnd crunch,NaN,NaN,"Snacks, Sweet snacks, Bars, Cereal bars",NaN,NaN,NaN,NaN,United States,"Roasted almonds, roasted peanuts, sugar, sunfl...",NaN,1 bar (35 g) (35 g),35.0,0.0,0.0,0.0,14.0,d,3.0,Sugary snacks,Biscuits and cakes,"en:to-be-completed, en:nutrition-facts-complet...",GENERAL MILLS SALES INC.,NaN,NaN,en:cereal-bars,NaN,NaN,NaN,NaN,543.0,2272.0,40.00,4.29,NaN,NaN,0.0,0.0,40.00,20.00,8.6,17.14,1.1425,0.457,0.0,0.0,NaN,NaN,NaN,0.057,0.00103,14.0
20827,Sparkling Lime Water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:ca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-complet...",NaN,NaN,NaN,NaN,NaN,NaN,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,0.0,0.0,0.00,0.00,NaN,NaN,NaN,NaN,0.00,0.00,NaN,0.00,0.0000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


On supprime les lignes qui sont dupliquées comme on le voit ci-dessous:

In [199]:
df[df.duplicated()]

,product_name,quantity,brands,categories,labels,labels_tags,labels_en,stores,countries,ingredients_text,allergens,serving_size,serving_quantity,additives_n,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,brand_owner,ecoscore_score_fr,ecoscore_grade_fr,main_category,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,vitamin-a_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-pp_100g,potassium_100g,calcium_100g,iron_100g,nutrition-score-fr_100g
109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49882,Premium popcorn,NaN,Pop-Secret,"Snacks, Sweet snacks, Popcorn",NaN,NaN,NaN,NaN,United States,"Whole grain popcorn kernels, palm oil, salt, a...",NaN,2 Tbsp (32 g),32.0,1.0,0.0,0.0,NaN,NaN,4.0,unknown,unknown,"en:to-be-completed, en:nutrition-facts-complet...","Diamond Foods, Inc.",60.0,b,en:popcorn,NaN,NaN,NaN,NaN,562.0,2351.0,34.38,18.75,12.50,4.69,0.00,0.000,50.00,NaN,6.2,6.25,2.9700,1.188,NaN,NaN,NaN,NaN,NaN,NaN,0.00112,NaN


In [200]:
df.drop_duplicates(inplace=True)
df.shape

(49167, 52)

On va maintenat traduire tout la liste des ingredients dans une même langue pour simplifier le traitement -> en français

On garde que les lignes qui ne sont pas vide dans la colonne ingredient_text

In [201]:
df = df[df.ingredients_text != ""]
df.dropna(subset=['ingredients_text'], how='all', inplace=True)

on n'a plus aucune ligne qui est soit vide ou nulle pour la colonne ingredients_text, on peut passer maintenant à la traduction

In [202]:
!pip install deep-translator
!pip install re

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [203]:
df.ingredients_text.isna().sum()
df.shape[0]

38127

In [218]:
import re
ingredients_text = df.ingredients_text.tolist()
ingredients_text = [re.sub('[^a-zA-Z0-9]+', ' ', e) for e in ingredients_text]
#ingredients_text = ingredients_text.str.replace(['1','2','3','4','5','6','7','8','9','9'],'')

In [221]:
from deep_translator import GoogleTranslator
ingredients_textSplitted = [ text[:4999] if(len(text) >= 4999) else text  for text in ingredients_text ]
ingredients_textSplitted
#translated = GoogleTranslator(source='auto', target='fr').translate(ingredients_textSplitted)  


['leche semidesnatada azucar 6 9 leche desnatada en polva fermentos lacticos contiene lactobacidus casei vitaminas b6 y d ',
 'eau graines de teguments de moutarde vinaigre de vin rouge sel vin rouge sucre mo t de raisin 6 2 oignons colorants extraits de carotte et extrait de paprika huile de tournesol son de moutarde sel cette moutarde uniquement disponible chez courte paille ',
 'antioxydant erythorbate de sodium colorant caramel origine ue tomate 33 3 mayonnaise 11 1 huile de colza 78 9 eau jaunes d oeuf 6 vinaigre moutarde eau graines de moutarde sel vinaigre curcuma sel dextrose stabilisateur gomme de cellulose conservateur sorbate de potassium colorant carot ne ar me ',
 'lait entier sucre amidon de ma s cacao agar agar ',
 'baguette poite vin pain baguette 50 6 farine de ble eau sel levure gluten farine de ble maite levure desactivee acide ascorbique garniture fromage mi ch vre 46 lait pasteurise 95 0 lait de vache 5 0 lait de ch vre sel ferments lactiques et daffinage coagulant

Suppression des lignes contenant la mention "unknown" dans les colonnes ci-dessous :

In [219]:
df = df[(df.pnns_groups_1 != "unknown") | (df.pnns_groups_2 != "unknown")]

In [220]:
df.shape

(24191, 52)